In [44]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [31]:
df = pd.read_csv('IMDB Dataset.csv')
df = df.iloc[0:5000]
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
4995,An interesting slasher film with multiple susp...,negative
4996,i watched this series when it first came out i...,positive
4997,Once again Jet Li brings his charismatic prese...,positive
4998,"I rented this movie, after hearing Chris Gore ...",negative


In [32]:
#lower casing
df['review'] = df['review'].apply(lambda review: review.lower())

#html tag removal
df['review'] = df['review'].apply(lambda review: BeautifulSoup(review, 'html.parser').get_text())

#url removal 
pattern = r'http[s]?://\S+|www\.\S+'
df['review'] = df['review'].apply(lambda review: re.sub(pattern, '', review))

df.head(10)

C:\Users\DELL\AppData\Local\Temp\ipykernel_9736\2544509236.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['review'] = df['review'].apply(lambda review: BeautifulSoup(review, 'html.parser').get_text())


,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
5,"probably my all-time favorite movie, a story o...",positive
6,i sure would like to see a resurrection of a u...,positive
7,"this show was an amazing, fresh & innovative i...",negative
8,encouraged by the positive comments about this...,negative
9,if you like original gut wrenching laughter yo...,positive


In [33]:
# remove punctuations
df['review'] = df['review'].apply(lambda review: review.translate(str.maketrans('', '', string.punctuation)))

#remove stop words
def remove_stopwords(text):
    stopwords_list = set(stopwords.words('english'))
    filtered_words = []
    for word in text.split():
        if(word not in stopwords_list):
            filtered_words.append(word)
    return ' '.join(filtered_words)


df['review'] = df['review'].apply(remove_stopwords)


df.head(10)

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive
5,probably alltime favorite movie story selfless...,positive
6,sure would like see resurrection dated seahunt...,positive
7,show amazing fresh innovative idea 70s first a...,negative
8,encouraged positive comments film looking forw...,negative
9,like original gut wrenching laughter like movi...,positive


In [34]:
# tokeization
df['review'] = df['review'].apply(lambda review: word_tokenize(review))
df.head(10)

,review,sentiment
0,"[one, reviewers, mentioned, watching, 1, oz, e...",positive
1,"[wonderful, little, production, filming, techn...",positive
2,"[thought, wonderful, way, spend, time, hot, su...",positive
3,"[basically, theres, family, little, boy, jake,...",negative
4,"[petter, matteis, love, time, money, visually,...",positive
5,"[probably, alltime, favorite, movie, story, se...",positive
6,"[sure, would, like, see, resurrection, dated, ...",positive
7,"[show, amazing, fresh, innovative, idea, 70s, ...",negative
8,"[encouraged, positive, comments, film, looking...",negative
9,"[like, original, gut, wrenching, laughter, lik...",positive


In [43]:
# Stemming
stemmer = SnowballStemmer('english')

def stem(wordsList):
    lemmatizedWords = []
    for word in wordsList:
        lemmatizedWords.append(spacyLemmatizer(word))
    return lemmatizedWords

df['review'] = df['review'].apply(stem)

df.head(10)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.